In [2]:
DATASET_FILE_PATH = "D:/Innovation/Dataset/AUDCAD_Candlestick_1_Hour_BID_2006-03-20_2025-03-25.csv"

In [3]:
SAVE_FILE_PATH = "D:/Innovation/Dataset/AUDCAD_Dataset.csv"

In [5]:
import pandas as pd
import numpy as np

# Read the dataset (adjust the path if necessary)
df = pd.read_csv(DATASET_FILE_PATH)

# Convert 'Gmt time' to datetime and split into separate date and time columns
df['Gmt time'] = pd.to_datetime(df['Gmt time'])
df['date'] = df['Gmt time'].dt.date
df['time'] = df['Gmt time'].dt.time

# Optionally, drop the original 'Gmt time' column
df = df.drop(columns=['Gmt time'])

# --- RSI14 Calculation ---
# Assumes the existence of a 'Close' column.
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# Calculate rolling means for gains and losses over 14 periods
avg_gain = gain.rolling(window=14, min_periods=14).mean()
avg_loss = loss.rolling(window=14, min_periods=14).mean()

rs = avg_gain / avg_loss
df['rsi14'] = 100 - (100 / (1 + rs))

# --- SMA Calculations ---
df['sma20'] = df['Close'].rolling(window=20, min_periods=20).mean()
df['sma50'] = df['Close'].rolling(window=50, min_periods=50).mean()
df['sma200'] = df['Close'].rolling(window=200, min_periods=200).mean()

# --- ADX Calculation ---
# Assumes the existence of 'High', 'Low', and 'Close' columns.
period = 14

# Calculate True Range (TR)
df['H-L'] = df['High'] - df['Low']
df['H-PC'] = (df['High'] - df['Close'].shift(1)).abs()
df['L-PC'] = (df['Low'] - df['Close'].shift(1)).abs()
df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)

# Calculate directional movements
df['up_move'] = df['High'] - df['High'].shift(1)
df['down_move'] = df['Low'].shift(1) - df['Low']

df['+DM'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
df['-DM'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)

# Smooth the TR and DM values by summing over the period
df['TR14'] = df['TR'].rolling(window=period, min_periods=period).sum()
df['+DM14'] = df['+DM'].rolling(window=period, min_periods=period).sum()
df['-DM14'] = df['-DM'].rolling(window=period, min_periods=period).sum()

# Calculate the directional indicators
df['+DI14'] = 100 * (df['+DM14'] / df['TR14'])
df['-DI14'] = 100 * (df['-DM14'] / df['TR14'])

# Compute DX and then ADX
df['DX'] = 100 * (abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14']))
df['ADX'] = df['DX'].rolling(window=period, min_periods=period).mean()

# Drop intermediate columns used for ADX calculation
df.drop(columns=['H-L', 'H-PC', 'L-PC', 'TR', 'up_move', 'down_move', '+DM', '-DM',
                 'TR14', '+DM14', '-DM14', '+DI14', '-DI14', 'DX'], inplace=True)

# --- CCI Calculation ---
# Using a period of 20
cci_period = 20
df['Typical Price'] = (df['High'] + df['Low'] + df['Close']) / 3
df['SMA_TP'] = df['Typical Price'].rolling(window=cci_period, min_periods=cci_period).mean()
df['Mean Deviation'] = df['Typical Price'].rolling(window=cci_period, min_periods=cci_period)\
    .apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
df['CCI'] = (df['Typical Price'] - df['SMA_TP']) / (0.015 * df['Mean Deviation'])

# Drop intermediate columns for CCI
df.drop(columns=['Typical Price', 'SMA_TP', 'Mean Deviation'], inplace=True)

# --- MACD Calculation ---
# MACD = EMA(12) - EMA(26)
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema12 - ema26

# --- Drop null rows ---
df.dropna(inplace=True)

# --- Reorder columns: place 'date' and 'time' as the first columns ---
cols = df.columns.tolist()
new_order = ['date', 'time'] + [col for col in cols if col not in ['date', 'time']]
df = df[new_order]

# --- Save the enhanced dataset ---
output_file = SAVE_FILE_PATH
df.to_csv(output_file, index=False)

print("Enhanced dataset saved to:", output_file)


C:\Users\Stephen\AppData\Local\Temp\ipykernel_14592\3307934934.py:8: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Gmt time'] = pd.to_datetime(df['Gmt time'])


Enhanced dataset saved to: D:/Innovation/Dataset/AUDCAD_Dataset.csv
